# **Projeto 2 EBAC: SQL**

Aluno: [Jean Matheus](https://www.linkedin.com/in/jean-matheus-sm/)

---

# **Exploração e análise dos dados de um dataset com informações de clientes de um banco usando SQL**

## **Os dados:**

O dataset possui as informações dos clientes, separadas da seguinte forma:

* idade = idade do cliente
* genero = genêro do cliente (M: masculino, F: feminino)
* dependentes = número de dependetes do cliente
* escolaridade = nível de escolaridade do cliente
* estado_civil = estado civíl do cliente
* salario_anual = faixa de salário anual do cliente
* tipo_cartao = tipo de cartão do cliente
* qtd_produtos = quantidade de produtos comprado nos últimos 12 meses
* iteracoes_12m = quantidade de interações nos últimos 12 meses
* meses_inativo_12m = quantidade de meses que o cliente não utilizou o cartão
* limite_credito = limite de crédtio do cartão
* valor_transacoes_12m = valor das transações dos últimos 12 meses
* qtd_transacoes_12m = quantidade de transações dos últimos 12 meses

*A tabela usada nesse projeto foi criada a partir do **AWS Athena** com **S3 Bucket**, a versão do dataset utilizada pode ser encontrada [aqui](https://github.com/andre-marcos-perez/ebac-course-utils/blob/main/dataset/credito.csv).*

## **Explorando os dados:**

Agora vamos conhecer melhor o dataset que estamos usando, os comandos a seguir nos mostrarão informações que irão nos ajudar com essa exploração e entender como é nosso dataset:

**Quantos clientes o dataset possui?**

Query:
```sql
SELECT COUNT(*) FROM credito
```
Resposta: 

![Numero de linhas](https://github.com/jeanmatheuss/jean-EBAC-SQL/blob/main/qtd_linhas.png?raw=true)

**Quais são os tipos de dados em cada coluna?**

Query:
```sql
DESCRIBE credito
```
Resposta:

![Describe](https://github.com/jeanmatheuss/jean-EBAC-SQL/blob/main/infos_dataset.png?raw=true)

**Como são os dados?**

Query:
```sql
SELECT * FROM credito limit 10;
```
Resposta:

![Dez primeiras linhas](https://github.com/jeanmatheuss/jean-EBAC-SQL/blob/main/limit10.png?raw=true)

>Aqui vemos que algumas colunas possuem valores nulos ('na'), ou seja precisaremos tratar esses dados ao longo do nosso projeto para que não entre nas contas futuras.

**Quais os tipos de escolaridade no dataset?**

Query:
```sql
SELECT escolaridade, CAST(100 * COUNT(*) / SUM(COUNT(*)) OVER() as DECIMAL(10,2)) as porcentagem
FROM credito
GROUP BY escolaridade;
```

![Porcentagem Escolaridade](https://github.com/jeanmatheuss/jean-EBAC-SQL/blob/main/percent_escol.png?raw=true)

>Aqui podemos ver que os diferentes tipos de escolaridade do dataset e ver que há 13,00% dos nossos dados possuem valores nulos ('na') para escolaridade.

**Quais os tipos de estado civíl no dataset?**

Query:
```sql
SELECT estado_civil, CAST(100 * COUNT(*) / SUM(COUNT(*)) OVER() as DECIMAL(10,2)) as porcentagem
FROM credito
GROUP BY estado_civil;
```

![Porcentagem EstadoCivil](https://github.com/jeanmatheuss/jean-EBAC-SQL/blob/main/percent_civil.png?raw=true)

>Aqui vemos os diferentes tipos de estado civíl e há apenas 4% dos dados nulos ('na').

**Quais os tipos de cartão no dataset?**

Query:
```sql
SELECT COUNT(*) AS QTD, tipo_cartao 
FROM credito 
GROUP BY tipo_cartao
```
![contagem cartao](https://github.com/jeanmatheuss/jean-EBAC-SQL/blob/main/count_cartao.png?raw=true)

>Aqui temos a contagem e os tipos de cartões respectivamente.

**Quais as faixas de salário no dataset?**

Query:
```sql
SELECT COUNT(*) AS QTD, salario_anual FROM credito GROUP BY salario_anual
```

![contagem salario](https://github.com/jeanmatheuss/jean-EBAC-SQL/blob/main/salario_count.png?raw=true)

>Aqui notamos que há uma valores nulos ('na') e precisaremos ficar atento durante a análise.

## **Análise de dados:**

Após a exploração do nosso dataset e entender com o que estamos trabalhando. Podemos agora fazer nossa análise e entender como são os clientes desse dataset e assim pensar em algumas conclusões a partir de algumas perguntas:

**Quantos clientes são homens e quantos são mulheres?**

Query:
```sql
SELECT COUNT(*) AS QTD, genero 
FROM credito 
GROUP BY genero;
```

![contagem genero](https://github.com/jeanmatheuss/jean-EBAC-SQL/blob/main/count_genero.png?raw=true)


![grafico genero](https://raw.githubusercontent.com/jeanmatheuss/jean-EBAC-SQL/main/grafico_genero.png)

**Qual a representação da faixa salarial dos clientes?**

Query:
```sql
SELECT salario_anual, CAST(100 * COUNT(*) / SUM(COUNT(*)) OVER() as DECIMAL(10,2)) as porcentagem
FROM credito8
GROUP BY salario_anual;
```

![percentual salario](https://github.com/jeanmatheuss/jean-EBAC-SQL/blob/main/salario_percent1.png?raw=true)

>Aqui podemos ver que os clientes possuem cerca de 27% dos clientes possuem renda abaixo de $40K, e 9% são valores nulos e apenas 9% recebem $120K ou mais.

Separando agora por genero e sem os dados nulos:
```sql
SELECT salario_anual, genero, CAST(100 * COUNT(*) / SUM(COUNT(*)) OVER() as DECIMAL(10,2)) as porcentagem
FROM credito
WHERE salario_anual != 'na'
GROUP BY salario_anual, genero;
```

![percentual salario genero](https://github.com/jeanmatheuss/jean-EBAC-SQL/blob/main/salario_percent_sexo.png?raw=true)

>Podemos ver que as mulheres desse banco ganham $60K ou menos e elas representam 33% dos clientes que informaram o salário.

[apagar: uma forma para atrair clientes mulheres que recebem mais de $60K? focar nos clientes que recebem $40K ou menos pois são a maioria no dataset]

**Quala a faixa de idade dos clientes desse banco?**

Query:
 ```sql
 SELECT ROUND(AVG(idade),2) AS media_idade, MIN(idade) as min_idade, MAX(idade) as max_idade, genero
 FROM credito
 GROUP BY genero;
```
![media idade](https://github.com/jeanmatheuss/jean-EBAC-SQL/blob/main/avg_idade.png?raw=true)

>Aqui notamos que as idades entre os homens e as mulheres são bem parecidas, logo não podemos afirmar nada sobre isso ter algum efeito.

**Qual a menor e maior transação?**
Query:
```sql
SELECT MIN(valor_transacoes_12m) AS min_transacao, MAX(valor_transacoes_12m) AS max_transacao
FROM credito;
```
![minimo e maximo](https://github.com/jeanmatheuss/jean-EBAC-SQL/blob/main/min_transacao.png?raw=true)

>As transações somada ao longo de 12 meses desse dataset estão entre 510,16 e 4776,58.

**Quais são as características dos cliente com os maiores limites?**
Query:
```sql
SELECT MAX(limite_credito) as limite_credito, escolaridade, tipo_cartao, genero
FROM credito
WHERE escolaridade != 'na' and tipo_cartao != 'na'
GROUP BY escolaridade, tipo_cartao, genero
ORDER BY limite_credito desc limit 10;
```
![limite credito](https://github.com/jeanmatheuss/jean-EBAC-SQL/blob/main/limite_credito.png?raw=true)

>Podemos notar que o limite de crédito não tem relação direta com a escolaridade (o maior limite de crédito é de alguém que não possuí educação formal). Entre os maiores limites há todos os tipos de cartões logo não há uma relação. Há a predominância de homens entre os maiores limites.

**Quais são as características dos cliente com os menores limites?**
```sql
SELECT MAX(limite_credito) as limite_credito, escolaridade, tipo_cartao, genero
FROM credito
WHERE escolaridade != 'na' and tipo_cartao != 'na'
GROUP BY escolaridade, tipo_cartao, genero
ORDER BY limite_credito asc limit 10;
```
![limite credito min](https://github.com/jeanmatheuss/jean-EBAC-SQL/blob/main/limite_credito_min.png?raw=true)

>Podemos notar algumas relações olhando para os menores limites tais como o tipo de cartão predominante é o blue, com apenas alguns silver e gold e nenhum platinum. A maiorias dos clientes com os menores limites são mulheres, o que provavelmente está relacionado ao salário das mulheres desse banco que são menores que os dos homens.

**Qu



Query:
```sql


```